# Requêtes APIs Orion

Bienvenue dans la documentation de ce notebook de test d'APIs permettant de récupérer des données utiles pour la création d'un score de risque pour les feux de forêt.


## Introduction

Les feux de forêt sont l'un des fléaux les plus dévastateurs pour les écosystèmes et les communautés qui dépendent d'eux. Dans de nombreux cas, la prévention est la meilleure solution, et cela nécessite des outils de prévision précis pour mesurer le risque de déclenchement de feux de forêt.

Ce notebook a pour objectif de tester différentes APIs pour créer un score de risque de feux de forêt en utilisant des informations telles que la météo, la végétation, l'humidité des sols, la topographie, et d'autres données environnementales. En explorant différentes sources de données, nous espérons trouver les meilleures informations pour créer un modèle de prédiction précis qui aidera les pompiers à prendre des mesures pour prévenir les feux de forêt.

Nous avons sélectionné quelques APIs mentionnées dans le papier [Wildfire Danger Prediction and Understanding With Deep Learning](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2022GL099368) et nous allons explorer chacune d'entre elles en détail. Cette documentation servira de guide pour comprendre comment requêter et utiliser ces APIs.

Nous espérons que ce notebook sera utile pour les personnes qui cherchent à améliorer la prévention des feux de forêt et à protéger nos écosystèmes.


### Comment commencer ?
#### Clonage du repo
Commencer par ouvrir son IDE (Pycharm, VS, terminal...), et cloner le repository Git : 
```
git clone git@github.com:pyronear/pyro-risks.git
```

/!\ Il faut que votre clé SSH soit configurée dans Github. Si ce n'est pas fait, voir [ici](https://docs.github.com/en/authentication/connecting-to-github-with-ssh/adding-a-new-ssh-key-to-your-github-account).

#### Environnement virtuel
Activer son [environnement virtuel](https://realpython.com/python-virtual-environments-a-primer/) :
```bash
source venv/bin/activate
```

/!\ Si vous n'avez pas configuré votre environnement virtuel, rendez vous à la racine du repo, et tapez dans votre terminal : 
```bash
python -m venv venv
pip install -r requirements.txt
```

#### Notebook
Lancez ensuite votre notebook :

```bash
pip install jupyter
jupyter notebook
```

Une fenêtre s'ouvre dans votre navigateur, c'est le notebook !
 
Se rendre dans `pyro_risks/notebooks`, puis vous pouvez créer votre propre notebook, ou bien lancer un notebook déjà créé.

## Requête des APIs mentionnées dans le papier

Le papier mentionne 6 datasets : 

 - **Daily weather data**, de **[ERA-5 Land](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview)** (Muñoz-Sabater et al., 2021) (température maximale à 2 m, vitesse maximale du vent, humidité relative minimale, précipitations totales, température maximale du point de rosée à 2 m et la pression maximale en surface), aggrégées à la journée.

 - **Satellite variables** de **[MODIS](https://www.earthdata.nasa.gov/learn/find-data/near-real-time/firms/mcd14dl-nrt)**, dont Normalized Difference Vegetation Index (NDVI; Didan, 2015), day and night Land Surface Temperature (LST; Wan et al., 2015).

 - **Soil moisture index** de **[European Drought Observatory](https://edo.jrc.ec.europa.eu/edov2/php/index.php?id=1000)** (Cammalleri et al., 2017).

 - **Roads distance, waterway distance, and yearly population density** de **WorldPop** (Tatem, 2017).

 - **Elevation and Slope** de **[Copernicus EU-DEM](https://land.copernicus.eu/imagery-in-situ/eu-dem/eu-dem-v1.1)** (Bashfield & Keim, 2011).

 - **Ten variables with the fraction of classes** (?) de **[Copernicus Corine Land Cover](https://land.copernicus.eu/pan-european/corine-land-cover)** (Büttner, 2014).


### 1. Daily weather data - ERA-5 Land
#### Requête
Documentation [ici](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation).

In [3]:
# TROUBLESHOOT : Jupyter ne reconnaissait pas mon venv donc j'ai dû réinstaller mes packages ici :
"""
%pip install geopandas
%pip install xarray
%pip install netCDF4
%pip install imblearn
%pip install xgboost
%pip install dvc
%pip install plot_metric
"""

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
  Using cached plot_metric-0.0.6-py3-none-any.whl (13 kB)
  Using cached matplotlib-3.7.0-cp310-cp310-macosx_11_0_arm64.whl (7.3 MB)
  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)


Note: you may need to restart the kernel to use updated packages.


On vérifie que l'on se trouve à la racine du répertoire pour lancer le code. Si ce n'est pas le cas, on se rend à la racine du répertoire :

In [1]:
import os
if os.getcwd().split("/")[-1]=="notebooks":
    os.chdir("../../")

On importe ici le package de l'API Copernicus Climate Data Store (CDS), et nos identifiants personnels pour effectuer des requêtes. Ces identifiants peuvent être créés sur le site [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/user/login?destination=/api/). 

Ensuite, il s'agit de configurer votre PC pour faire des requêtes sans rentrer votre nom d'utilisateur et votre mot de passe dans le code. Pour ce faire, se référer à la [documentation suivante](https://cds.climate.copernicus.eu/api-how-to), ou à la [vidéo suivante](https://www.youtube.com/watch?v=AmF1nn7o6Hc&ab_channel=ClimateUnboxed). 

Ces identifiants vont être récupérés directement dans le code.

In [23]:
import cdsapi
import xarray as xr
import pandas as pd
import geopandas as gpd

Téléchargement des données pour une journée :

In [17]:
c = cdsapi.Client()
year=2021
month=12
#day=['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
day=1
time=['00:00','01:00','02:00','03:00','04:00','05:00','06:00','07:00','08:00','09:00','10:00','11:00','12:00','13:00','14:00','15:00','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00']
output_path = os.getcwd()+"/pyro_risks/notebooks/datastore"
file_path = os.path.join(output_path, f"era5land_{year}_{month}_{day}.nc")

# Requête des valeurs maximales
c.retrieve(
    "reanalysis-era5-land",
    {
        "variable": [
            "2m_temperature",
            "2m_dewpoint_temperature",
            "surface_pressure",
            "10m_u_component_of_wind",
            "10m_v_component_of_wind",
            "total_precipitation",
        ],
        "year": year,
        "month": month,
        "day": day,
        "time":time,
        "area": [
            51,
            -6,
            41,
            10,
        ],
        "format": "netcdf",
    },
    file_path,
)

2023-02-26 19:31:06,558 INFO Welcome to the CDS
2023-02-26 19:31:06,560 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-02-26 19:31:06,647 INFO Request is queued
2023-02-26 19:31:09,256 INFO Request is running
2023-02-26 19:31:39,196 INFO Request is completed
2023-02-26 19:31:39,199 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data2/adaptor.mars.internal-1677436290.7642412-31057-16-25309392-4bfe-4fe1-85f2-b0b6ede22f2b.nc to /Users/camillemodeste/Documents/Workflow/Pyronear/pyro-risks/pyro_risks/notebooks/datastore/era5land_2021_12_1.nc (4.5M)
2023-02-26 19:31:39,875 INFO Download rate 6.6M/s                                                                                                                                                                        


Result(content_length=4686664,content_type=application/x-netcdf,location=https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data2/adaptor.mars.internal-1677436290.7642412-31057-16-25309392-4bfe-4fe1-85f2-b0b6ede22f2b.nc)

#### Exploration du dataset et transformation des données

In [20]:
ds = xr.open_dataset(file_path)
data = ds.to_dataframe()

# Drop NaNs (nulle part)
data = data.dropna()
data = data.reset_index()

data["time"] = pd.to_datetime(
    data["time"], format="%Y-%m-%d %H:%M:%S", errors="coerce"
)
data["time"] = data["time"].dt.normalize()

In [21]:
# Visualisation de la donnée à ce stade
data

,longitude,latitude,time,t2m,d2m,sp,u10,v10,tp
0,-6.0,43.500000,2021-12-01,283.223846,278.238007,99758.531250,2.852300,1.843215,4.217960e-06
1,-6.0,43.500000,2021-12-01,283.360474,278.680450,99717.445312,3.519376,1.826164,0.000000e+00
2,-6.0,43.500000,2021-12-01,283.532257,279.322113,99635.281250,3.575446,2.544003,0.000000e+00
3,-6.0,43.500000,2021-12-01,283.975250,280.023224,99560.046875,3.808110,2.437265,9.369105e-07
4,-6.0,43.500000,2021-12-01,284.269867,280.547394,99505.109375,4.241998,2.717239,1.874752e-06
...,...,...,...,...,...,...,...,...,...
265363,10.0,44.099998,2021-12-01,284.175568,282.163879,98389.968750,5.570125,4.847565,5.778103e-03
265364,10.0,44.099998,2021-12-01,284.308105,282.391144,98340.968750,6.406198,4.649434,7.094313e-03
265365,10.0,44.099998,2021-12-01,284.379974,282.733002,98276.625000,6.648294,4.605103,8.718014e-03
265366,10.0,44.099998,2021-12-01,284.438599,283.023621,98227.132812,6.632835,4.624882,1.067686e-02


Je regarde si j'ai bien 24x chaque point (utile pour + tard) :

In [32]:
data.groupby(['longitude', 'latitude']).size().max()

24

C'est bien le cas !

Il s'agit désormais d'aggréger nos données. On les aggrège par point, car chaque point apparaît ici 24x pour les 24h de la journée.

Il est stipulé dans le papier que l'on veut :
 - maximum 2 m temperature => aggrégation "max"
 - maximum wind speed (donnée absente, à calculer)
 - maximum 2 m dewpoint temperature => aggrégation "max"
 - maximum surface pressure => aggrégation "max"
 - minimum relative humidity (donnée absente, à calculer)
 - total precipitation => aggrégation "sum"

Donc on va devoir faire de multiples aggrégations, et d'autres transformations de données ...

In [85]:
# Aggrégation pour avoir les données max d'un point sur 24h
data_aggregation_max = data.drop(columns=['tp'], inplace=False)
data_aggregation_max = data_aggregation_max.groupby(['longitude', 'latitude','time'], as_index=False).max()
new_col_names=['longitude', 'latitude','date']
for col_name in data_aggregation_max.columns[3:]:
    new_col_name = col_name+"_max"
    new_col_names.append(new_col_name)
data_aggregation_max.columns = new_col_names

# Aggrégation pour avoir les données totales d'un point sur 24h
data_aggregation_sum = data.drop(columns=['time','t2m','d2m','sp','u10','v10'], inplace=False)
data_aggregation_sum = data_aggregation_sum.groupby(['longitude', 'latitude'], as_index=False).sum(numeric_only=True)
new_col_names=['longitude', 'latitude']
for col_name in data_aggregation_sum.columns[2:]:
    new_col_name = col_name+"_sum"
    new_col_names.append(new_col_name)
data_aggregation_sum.columns = new_col_names

## TODO: minimum relative humidity (donnée absente, à calculer)

# On reconsolide
data_aggregated = pd.merge(data_aggregation_max, data_aggregation_sum,  how='left', on=['longitude', 'latitude'])

/var/folders/gv/tzh6ct3d489cjz8_3rtb6vbm0000gn/T/ipykernel_37843/875651731.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  data_aggregation_max = data_aggregation_max.groupby(['longitude', 'latitude','time'], as_index=False).max()


In [86]:
data_aggregated

,longitude,latitude,date,t2m_max,d2m_max,sp_max,u10_max,v10_max,tp_sum
0,-6.0,41.000000,2021-12-01,280.985504,280.187164,93516.617188,4.476497,1.624624,0.011097
1,-6.0,41.099998,2021-12-01,280.877960,280.212311,93583.437500,4.280252,1.732726,0.011070
2,-6.0,41.200001,2021-12-01,280.573608,280.013092,93120.648438,4.235186,1.857538,0.010432
3,-6.0,41.299999,2021-12-01,280.321289,279.757294,92814.765625,4.321125,1.933243,0.009082
4,-6.0,41.400002,2021-12-01,280.459961,279.868500,93475.539062,4.507676,2.080562,0.007818
...,...,...,...,...,...,...,...,...,...
11052,10.0,50.599998,2021-12-01,278.315918,277.789276,94062.554688,5.538159,5.268037,0.094508
11053,10.0,50.700001,2021-12-01,278.687012,278.097778,95207.390625,5.589776,5.324646,0.089392
11054,10.0,50.799999,2021-12-01,278.993896,278.293121,96108.710938,5.708990,5.293613,0.086839
11055,10.0,50.900002,2021-12-01,279.196777,278.439148,96641.781250,5.797811,5.193014,0.087047


#### Transformation en dataframe geopandas

In [87]:
geo_data_aggregated = gpd.GeoDataFrame(
    data_aggregated,
    geometry=gpd.points_from_xy(data_aggregated["longitude"], data_aggregated["latitude"]),
    crs="EPSG:4326",
)
geo_data_aggregated = geo_data_aggregated.drop(['longitude', 'latitude'], axis=1)

In [88]:
# Visualisation de la donnée à ce stade
geo_data_aggregated

,date,t2m_max,d2m_max,sp_max,u10_max,v10_max,tp_sum,geometry
0,2021-12-01,280.985504,280.187164,93516.617188,4.476497,1.624624,0.011097,POINT (-6.00000 41.00000)
1,2021-12-01,280.877960,280.212311,93583.437500,4.280252,1.732726,0.011070,POINT (-6.00000 41.10000)
2,2021-12-01,280.573608,280.013092,93120.648438,4.235186,1.857538,0.010432,POINT (-6.00000 41.20000)
3,2021-12-01,280.321289,279.757294,92814.765625,4.321125,1.933243,0.009082,POINT (-6.00000 41.30000)
4,2021-12-01,280.459961,279.868500,93475.539062,4.507676,2.080562,0.007818,POINT (-6.00000 41.40000)
...,...,...,...,...,...,...,...,...
11052,2021-12-01,278.315918,277.789276,94062.554688,5.538159,5.268037,0.094508,POINT (10.00000 50.60000)
11053,2021-12-01,278.687012,278.097778,95207.390625,5.589776,5.324646,0.089392,POINT (10.00000 50.70000)
11054,2021-12-01,278.993896,278.293121,96108.710938,5.708990,5.293613,0.086839,POINT (10.00000 50.80000)
11055,2021-12-01,279.196777,278.439148,96641.781250,5.797811,5.193014,0.087047,POINT (10.00000 50.90000)


#### Avantages/Inconvénients de ce dataset : 

Avantages / Inconvénients :
- manque l'humidité relative
 - manque le calcul du vent max (à creuser, voir ce que u et v veulent dire etc, est-ce qu'il faudrait pas un angle en plus pour avoir une vraie vitesse ...)
 - difficile à requêter, je n'ai étudié que 1 jour car au-delà l'API soulevait une erreur (trop de données requêtées)
 
 
 => Solution ici = télécharger le dataset en dur sur un drive, si on veut jouer avec plein de données

### 2. Satellite variables - MODIS

### 3. Soil moisture index - European Drought Observatory

### 4. Roads distance, waterway distance, and yearly population density - WorldPop

### 5. Elevation and Slope - Copernicus EU-DEM

### 6. Ten variables with the fraction of classes - Copernicus Corine Land Cover